# Use GPU for faster results

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
from PIL import Image
from tqdm import tqdm
import pandas as pd
import os

df2 = pd.read_csv('/kaggle/input/semeval/SemEval2025/test.tsv', sep="\t")
ipath2 = []
sentences2 = []

for i, row in df2.iterrows():
    compound = row['compound']
    expected_order = row['expected_order']
    ch = "".join(char for char in compound if char != " ")
    base_path = f'/kaggle/input/semeval/SemEval2025/train/{ch}'

    if os.path.exists(base_path):
        expected_order = expected_order.strip("[]").replace("'", "").split(",") 

        for img_filename in expected_order:
            img_filename = img_filename.strip()
            full_path = os.path.join(base_path, img_filename)
            if os.path.isfile(full_path):
                ipath2.append(full_path)
            else:
                print(f"File not found: {full_path}")
    else:
        print(f"Directory not found: {base_path}")
    sentences2.append(row['sentence'])

In [2]:
import torch
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
new_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def calculate_topk_accuracy(k):
    new_clip_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i, sentence in enumerate(tqdm(sentences2, desc=f"Evaluating Top-{k} Accuracy on Test Data")):
            text_inputs = new_processor(text=sentence, return_tensors="pt", padding=True, truncation=True).to(device)
            text_features = new_clip_model.get_text_features(
                input_ids=text_inputs["input_ids"], 
                attention_mask=text_inputs["attention_mask"]
            )
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            image_start_idx = 5 * i
            image_end_idx = image_start_idx + 4
            selected_image_paths = ipath2[image_start_idx:image_end_idx + 1]
            image_embeddings_list = []
            
            for path in selected_image_paths:
                image = Image.open(path).convert("RGB")  # Ensure image is in RGB mode
                pixel_values = new_processor(images=image, return_tensors="pt").pixel_values.to(device)
                image_features = new_clip_model.get_image_features(pixel_values=pixel_values)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                image_embeddings_list.append(image_features)

            image_embeddings = torch.cat(image_embeddings_list, dim=0)
            similarities = torch.matmul(text_features, image_embeddings.T)
            similarities = similarities.squeeze(0)
            ranked_indices = torch.argsort(similarities, descending=True)
            if 0 in ranked_indices[:k]:
                correct += 1

            total += 1

    topk_accuracy = correct / total * 100
    return topk_accuracy

top1_accuracy_new = calculate_topk_accuracy(1)
print(f"Top-1 Accuracy on Test Data (New Model): {top1_accuracy_new:.4f}%")
top2_accuracy_new = calculate_topk_accuracy(2)
print(f"Top-2 Accuracy on Test Data (New Model): {top2_accuracy_new:.4f}%")


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Evaluating Top-1 Accuracy on Test Data: 100%|██████████| 15/15 [00:05<00:00,  2.60it/s]


Top-1 Accuracy on Test Data (New Model): 53.3333%


Evaluating Top-2 Accuracy on Test Data: 100%|██████████| 15/15 [00:03<00:00,  4.32it/s]

Top-2 Accuracy on Test Data (New Model): 66.6667%
